In [124]:
import tensorflow as tf
from tensorflow.keras import layers, preprocessing, Sequential
import os

In [94]:
model = tf.keras.Sequential(
    [layers.InputLayer(input_shape=(256,256,3)),
    layers.experimental.preprocessing.Rescaling(1/255.),
     layers.Conv2D(16,(3,3), activation='relu'),
     layers.MaxPooling2D((2,2)),
     layers.Conv2D(32,(3,3), activation='relu'),
     layers.MaxPooling2D((2,2)),
     layers.Conv2D(64,(3,3), activation='relu'),
     layers.MaxPooling2D((2,2)),
     layers.Conv2D(128,(3,3), activation='relu'),
     layers.MaxPooling2D((2,2)),
     layers.Flatten(),
     layers.Dense(128, activation='relu'),
     layers.Dense(3, activation='softmax')
     ]
)

In [95]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                                         metrics=['accuracy'] )

In [96]:
def listdir_nohidden(path, jpg_only=False):
    if jpg_only:
        return sorted(
            [el for el in os.listdir(path) if not el.startswith(".") and (el.endswith(".jpg") or el.endswith(".JPG"))])
    else:
        return sorted([el for el in os.listdir(path) if not el.startswith(".")])

data_augmentation = Sequential([
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
            layers.experimental.preprocessing.RandomContrast((0, 1))])

def data_repartition(zord, directory_):
    folders = []
    if zord == "main_zord":
        classes = listdir_nohidden(directory_)
        for classe in classes:
            dir_ = directory_ + "/" + classe
            labels = listdir_nohidden(dir_)
            tot = 0
            for label in labels:
                tot += len(listdir_nohidden(diver(dir_ + "/" + label), jpg_only=True))
            folders.append(tot)

    else:
        for label in listdir_nohidden(directory_):
            file_nb = len(listdir_nohidden(diver(directory_ + "/" + label), jpg_only=True))
            folders.append(file_nb)

    return folders

def weighter(folders):
    m = max(folders)
    class_weight = {}
    for i in range(len(folders)):
        class_weight[i] = float(m) / float(folders[i])

    return class_weight

def diver(path):
    while len(listdir_nohidden(path))==1 :
        path+="/" + listdir_nohidden(path)[0]
    return path

In [97]:
directory_ = "/Users/lucas/swiss_knife/data/wheel"
train_ds = preprocessing.image_dataset_from_directory(
                directory_,
                labels="inferred",
                label_mode="int", shuffle=True, batch_size=32)

folders = data_repartition("wheel", directory_)

class_weight = weighter(folders)

print("As data is imbalanced, the following weights will be applied ", list(class_weight.values()))

print("Augmenting the train_ds")
augmented_train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
augmented_train_ds = augmented_train_ds.prefetch(buffer_size=32)  # facilitates training
augmented_train_ds.shuffle(1000)
print("Augmentation is done. Importation of InceptionV3 beginning...")

Found 1620 files belonging to 3 classes.
As data is imbalanced, the following weights will be applied  [1.365296803652968, 1.0, 1.023972602739726]
Augmenting the train_ds
Augmentation is done. Importation of InceptionV3 beginning...


In [98]:
model.fit(augmented_train_ds, epochs =5, class_weight=class_weight)

Epoch 1/10
51/51 [==============================] - 29s 577ms/step - loss: 0.8600 - accuracy: 0.6216

In [99]:
model.summary()

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_5 (Rescaling)      (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 30, 30, 64)      

In [104]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
                "/Users/lucas/swiss_knife/data",
                labels=['triple_resistance_coude', 'sonde', 'resistance_enroulee', 'resistance_grille','handle_lockless', 'handle_lock','white_wheel_ro', 'black_wheel', 'white_wheel_sq','big_evco', 'small_evco', '6_buttons', '4_buttons','ball_bearing','vn5', 'vn7','sofiane','ventilo_octo', 'ventilo_carre']
,
                label_mode="int", shuffle=True, batch_size=32)

ValueError: Expected the lengths of `labels` to match the number of files in the target directory. len(labels) is 19 while we found 9183 files in /Users/lucas/swiss_knife/data.

In [103]:
for el in os.walk("/Users/lucas/swiss_knife/data"):
    print(el)

['triple_resistance_coude', 'sonde', 'resistance_enroulee', 'resistance_grille','handle_lockless', 'handle_lock','white_wheel_ro', 'black_wheel', 'white_wheel_sq','big_evco', 'small_evco', '6_buttons', '4_buttons','ball_bearing','vn5', 'vn7','sofiane','ventilo_octo', 'ventilo_carre']

('/Users/lucas/swiss_knife/data', ['tube', 'handle', 'wheel', 'regulator', 'ball_bearing', 'motor', 'sofiane', 'ventilo'], ['.DS_Store'])
('/Users/lucas/swiss_knife/data/tube', ['triple_resistance_coude', 'sonde', 'resistance_enroulee', 'resistance_grille'], ['.DS_Store'])
('/Users/lucas/swiss_knife/data/tube/triple_resistance_coude', [], ['triple_resistance_coude_606.jpg', 'triple_resistance_coude_160.jpg', 'triple_resistance_coude_174.jpg', 'triple_resistance_coude_612.jpg', 'triple_resistance_coude_148.jpg', 'triple_resistance_coude_362.jpg', 'triple_resistance_coude_404.jpg', 'triple_resistance_coude_410.jpg', 'triple_resistance_coude_376.jpg', 'triple_resistance_coude_438.jpg', 'triple_resistance_coude_389.jpg', 'triple_resistance_coude_216.jpg', 'triple_resistance_coude_570.jpg', 'triple_resistance_coude_564.jpg', 'triple_resistance_coude_202.jpg', 'triple_resistance_coude_558.jpg', 'triple_resistance_coude_10.jpg', 'triple_resistance_coude_38.jpg', 'triple_resistance_coude_2.jpg

In [117]:
l=[['ball_bearing', 370], ['handle_lock', 385], ['handle_lockless', 377], ['vn5', 720], ['vn7', 625], ['4_buttons', 355], ['6_buttons', 453], ['big_evco', 300], ['small_evco', 366], ['sofiane', 318], ['resistance_enroulee', 865], ['resistance_grille', 640], ['sonde', 457], ['triple_resistance_coude', 634], ['ventilo_carre', 299], ['ventilo_octo', 399], ['black_wheel', 438], ['white_wheel_ro', 598], ['white_wheel_sq', 584]]
import numpy as np
tab = np.array(l)

tab[:,1].astype("int32")

array([370, 385, 377, 720, 625, 355, 453, 300, 366, 318, 865, 640, 457,
       634, 299, 399, 438, 598, 584], dtype=int32)

In [118]:
def listdir_nohidden(path, jpg_only=False):
    if jpg_only:
        return sorted(
            [el for el in os.listdir(path) if not el.startswith(".") and (el.endswith(".jpg") or el.endswith(".JPG"))])
    else:
        return sorted([el for el in os.listdir(path) if not el.startswith(".")])

def data_repartition(zord, directory_):
    folders = []
    if zord == "main_zord":
        classes = listdir_nohidden(directory_)
        for classe in classes:
            dir_ = directory_ + "/" + classe
            labels = listdir_nohidden(dir_)
            tot = 0
            for label in labels:
                tot += len(listdir_nohidden(diver(dir_ + "/" + label), jpg_only=True))
            folders.append(tot)

    else:
        for label in listdir_nohidden(directory_):
            file_nb = len(listdir_nohidden(diver(directory_ + "/" + label), jpg_only=True))
            folders.append(file_nb)

    return folders

def weighter(folders):
    m = max(folders)
    class_weight = {}
    for i in range(len(folders)):
        class_weight[i] = float(m) / float(folders[i])

    return class_weight

def diver(path):
    while len(listdir_nohidden(path))==1 :
        path+="/" + listdir_nohidden(path)[0]
    return path

def one4all_labeller(path):
    obj_map=[]
    folders = listdir_nohidden(path)
    for folder in folders :
        path_ = path+"/"+folder
        objects = listdir_nohidden(path_)
        for obj in objects :
            path__ = path_ +"/"+obj
            file_nb = len(listdir_nohidden(diver(path__), jpg_only=True))
            obj_map.append([obj,file_nb])
    return obj_map

In [128]:
dir = "/Users/lucas/swiss_knife"
dir+="/data"
print("one4all need to be trained...")
temp = one4all_labeller(dir)
tab = np.array(temp)
labels = tab[:,0]
print(labels)

folders = tab[:, 1].astype("int32")
class_weight = weighter(folders)
label_list= []
for j in range(len(folders)):
    for _ in range(folders[j]):
        label_list.append(labels[j])

assert len(label_list)==sum(folders), "didnt work"
print(label_list)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dir,
    label_mode=None)
print(train_ds.class_names)
print("As data is imbalanced, the following weights will be applied ", list(class_weight.values()))

one4all need to be trained...
['ball_bearing' 'handle_lock' 'handle_lockless' 'vn5' 'vn7' '4_buttons'
 '6_buttons' 'big_evco' 'small_evco' 'sofiane' 'resistance_enroulee'
 'resistance_grille' 'sonde' 'triple_resistance_coude' 'ventilo_carre'
 'ventilo_octo' 'black_wheel' 'white_wheel_ro' 'white_wheel_sq']
['ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'ball_bearing', 'bal